# REST

## R.1 학습내용

### 1) 목표

* API를 사용하여 열린데이터, 공공데이터포털 데이터를 가져올 수 있다.
* API로 가져온 xml, json 데이터를 분석가능한 형식으로 변환할 수 있다.
* nosql, 파일로 저장할 수 있다.
* 웹에서 사용되는 한글의 인코딩 방식을 사용할 수 있다.

### 2) 문제

* 문제 REST-1: ip주소에서 지역정보 알아내기
* 문제 REST-2: 키를 저장하고 읽기
* 문제 REST-3: 서울시 노선별 지하철역 목록 구하기 
* 문제 REST-4: 서울시 외국인 주민 자녀 국적취득 년도별 시군구 합계 구하기 
* 문제 REST-5: 2015년 서울시 지하철역별 월별 승하차인원 구하기
* 문제 REST-6: 2015년 1년 동안 지하철역 승하차 인원
* 문제 REST-7: '2015년 1년 동안 지하철역 승하차 인원' 저장하기
    * 파일, nosql 저장
* 문제 REST-8: 서울 7016번 광화문~상명대까지 2015년 승하차 인원 월별 정류장별 최대 최소 구하기
* 문제 REST-9: 서울시 종로구 관측소 현재 온도, 습도, 강수량 구하기
* 문제 REST-10: 공공데이터포털에서 제공하는 2015년 서울시 구별 교통사고 사망자 수 구하기
* 문제 REST-11: 공공데이터포털 대기 오염 정보
* 문제 REST-12: 공공데이터포털 서울특별시교육청 공공도서관 소장도서정보

### 3) 연습

* 광화문 지역 2015년 11월 하루 동안 시간대별 미세먼지량
* 공공데이터포털 의료빅데이터
* 연습 REST: pandas로 웹 데이터 야후 주식 읽기
* 연습 REST: etc
    * 다음
    * http://kosis.kr
    * 지오코딩 - T Map, 다음, 네이버, 구글 (구글맵에 사용)

## R.2 소개

### Open Data

* [Open](http://opendefinition.org/)이란 누구나 원하는대로 자유롭게 사용하고, 수정하고, 공유할 수 있는 개념을 말하는 것이고, Open Data는 그러한 데이터를 말한다. "Open data and content can be freely used, modified, and shared by anyone for any purpose"
* Open API (Application Programming Interface)는 오픈 데이터를 누구나 (보통 개발자) 사용할 수 있도록 제공되는 표준화된 api를 말한다.
* Web API - HTTP프로토콜을 사용하여 웹 데이터를 제공하는 API.

### REST

* REpresentational State Transfer는 '웹서비스'를 사용하기 위한 아키텍처를 말한다.
* '웹서비스'는 웹에서 제공하는 서비스, 자원을 가져오거나 저장하는 기능을 말한다.
* SOAP (Simple Object Access Protocol) 보다는 REST가 간편하여 많이 사용된다.
* '웹서비스'는 'http 프로토콜'을 통하여 '자원'을 사용할 수 있게 한다.
    * 자원 resource는 html pages, xml, videos 등을 말한다.
    * resource는 xml 또는 json으로 표현된다 (representation).
    * 클라이언트와 서버는 http request, http response 형식으로 데이터를 주고 받는다.
    * http 프로토콜과 같이 상태를 보관하지 않는 특성을 가진다 (stateless).
* REST uniform interface를 사용한다.
    * uri, 자원 id, 방법(HTTP의 post, delete, get, put)을 명시하여 resource를 읽어 온다.
    * 주로 XML, json 형식으로 정보를 제공한다.

### RESTful api

* api의 설계
* endpoint url을 지정해서 자원을 가져오고, 대량 데이터를 가져오는 경우 pging, rate limit를 주의한다.

http 방법 | 설명 | 예제
----------|----------|----------
GET | 자원을 읽어 온다. | http://httpbin.org/get
PUT | 새로운 자원을 입력하거나 갱신한다. | http://<url>/put (x) http://<url>/put/id (o)
POST | 새로운 자원을 입력하거나 갱신한다. | http://<url>/post (o)
DELETE | 자원을 삭제한다. | http://<url>/delete/id

### 열린데이터와 공공데이터포털

* 모두 공공데이터이고, Open Data이다.
    * 공공데이터포털은 포털형식으로 제공한다 (교통, 재난, 의료 등).
    * 열린데이터는 시군구별 공공데이터를 제공한다.

열린데이터 | 공공데이터포털
----------|----------
시군구별로 제공하는 Open Data | 정부가 제공, 여러 관공서에서 제공하는 Open Data의 포털
data.seoul.go.kr<br>data.busan.go.kr<br>data.jongno.go.kr | data.go.kr
'/'를 사용하여 api endpoint를 제공한다. | url에 query parameters를 추가해서 사용한다. '?'로 시작하여 key, value의 쌍을 '&'로 묶음 예: '?k1=v1&k2=v2'


* 예제
    * 서울시 열린데이터 data.seoul.go.kr의 지하철 역사정보
    * 공공데이터 포털 go.kr의 대기정보

In [4]:
%%writefile src/ds_rest_dataseoul_ex.py
import os
import requests
import urlparse

# (1) make params with resource IDs
KEY='73725974496a736c34395757705847'
TYPE='json'
SERVICE='SearchSTNBySubwayLineService'
START_INDEX=str(1)
END_INDEX=str(10)
LINE_NUM=str(2)
params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)

# (2) make a full url
_url='http://openAPI.seoul.go.kr:8088/'
url=urlparse.urljoin(_url,params)

# (3) get data
data=requests.get(url).text
print data[:300]

Overwriting src/ds_rest_dataseoul_ex.py


In [5]:
!python src/ds_rest_dataseoul_ex.py

{"SearchSTNBySubwayLineService":{"list_total_count":51,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"STATION_CD":"0201","STATION_NM":"시청","LINE_NUM":"2","FR_CODE":"201"},{"STATION_CD":"0202","STATION_NM":"을지로입구","LINE_NUM":"2","FR_CODE":"202"},{"STATION_CD":"0203","STATION_NM":"을지로3가"


In [6]:
%%writefile src/ds_rest_gokr_ex.py
import os
import requests
import urlparse
import urllib

# (1) service + operation
SERVICE='ArpltnInforInqireSvc'
OPERATION_NAME='getMinuDustFrcstDspth'
params1=os.path.join(SERVICE,OPERATION_NAME)

# (2) query params encoding
_d=dict()
_d['dataTerm']='month'
params2 = urllib.urlencode(_d)

# (3) add my service key
keygokr='8Bx4C1%2Bbogiu3i0M5k7aqOf%2BTA5yCWK%2BFDI7nTSJ0oAhDj1NC5CtbI1TMgFmnpuZxJ9LBoCScqBhuw5sul2tmQ%3D%3D'
params=params1+'?'+'serviceKey='+keygokr+'&'+params2

# (4) make a full url
_url='http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc'
url=urlparse.urljoin(_url,params)

# (5) get data

data=requests.get(url).text
print data[:300]

Overwriting src/ds_rest_gokr_ex.py


In [7]:
!python src/ds_rest_gokr_ex.py

## R.3 API 연습

* cURL 또는 python을 사용할 수 있다.
* [cURL](https://en.wikipedia.org/wiki/CURL)
    * 서버와 데이터를 교환하는 도구. 다양한 통신 프로토콜을 지원한다.
    * 리눅스 설치 apt install curl
    * 윈도우 설치
        * (cURL 다운로드)[https://curl.haxx.se/download.html]
        * 운영체제 선택 Win32 또는 Win64. 그리고 보통 윈도우면 Generic 선택.
        * unzip
        * PATH 설정 (C:\Windows\System32에 이동)
* 연습 사이트
    * http://httpbin.org
    * http://postbin.org


### postbin.org

* http:postbin.org post를 연습할 수 있는 사이트로, http://requestb.in/ 로 연결된다

* post에 이용될 resource는 임시로 만들어 준다.
* key, value로 입력할 데이터를 만들어 준다.

In [15]:
!curl -X POST -d "jsl=test" http://requestb.in/v6a6lhv6

ok

* python을 사용해서 입력한다.

In [19]:
import requests, time
r = requests.post('http://requestb.in/v6a6lhv6', data={"ts":time.time()})
print r.status_code
print r.content

200
ok


### httpbin.org

* (httpbin)[https://github.com/Runscope/httpbin] api를 테스트하기 좋은 사이트.
* postbin.org는 post 인터페이스만 제공한다.

In [35]:
!curl http://httpbin.org/ip

{
  "origin": "117.16.44.45"
}


In [36]:
!curl http://httpbin.org/get

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.47.0"
  }, 
  "origin": "117.16.44.45", 
  "url": "http://httpbin.org/get"
}


* param을 key,value 쌍으로 넘겨줄 수 있다.

In [37]:
!curl http://httpbin.org/get?show_env=1

{
  "args": {
    "show_env": "1"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Host": "httpbin.org", 
    "Runscope-Service": "httpbin", 
    "User-Agent": "curl/7.47.0", 
    "X-Forwarded-For": "117.16.44.45", 
    "X-Real-Ip": "117.16.44.45"
  }, 
  "origin": "117.16.44.45", 
  "url": "http://httpbin.org/get?show_env=1"
}


* python requests 라이브러리를 사용해 본다

In [40]:
r = requests.options("http://httpbin.org/get")

In [42]:
r.status_code

200

In [43]:
bad_r = requests.get('http://httpbin.org/status/404')

In [44]:
bad_r.status_code

404

In [45]:
r.headers

{'Content-Length': '0', 'Access-Control-Max-Age': '3600', 'Server': 'nginx', 'Connection': 'keep-alive', 'Allow': 'HEAD, OPTIONS, GET', 'Access-Control-Allow-Credentials': 'true', 'Date': 'Sun, 09 Oct 2016 01:47:03 GMT', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, PATCH, OPTIONS', 'Content-Type': 'text/html; charset=utf-8'}

In [46]:
r.headers['Content-Type']

'text/html; charset=utf-8'

* post, put, delete 서비스는 사용할 자원이 저장되어 있지 않으므로, 실행 결과는 의미가 없다.
    ```
    r = requests.post("http://httpbin.org/post")
    r = requests.put("http://httpbin.org/put")
    r = requests.delete("http://httpbin.org/delete")
    ```

In [47]:
r = requests.post("http://httpbin.org/post")

In [48]:
r.json()

{u'args': {},
 u'data': u'',
 u'files': {},
 u'form': {},
 u'headers': {u'Accept': u'*/*',
  u'Accept-Encoding': u'gzip, deflate',
  u'Content-Length': u'0',
  u'Host': u'httpbin.org',
  u'User-Agent': u'python-requests/2.11.1'},
 u'json': None,
 u'origin': u'117.16.44.45',
 u'url': u'http://httpbin.org/post'}

## R.4 한글

* 운영체제에서 사용하는 인코딩 방식
    * unicode를 사용하는 것이 문제를 줄일 수 있다.
    * locale을 'utf-8'로 정의한다.
* urllib을 사용할 경우
    * urlencode: urlopen()에 사용될 param을 만든다.
        * dict형식을 key=value, '&' 문자로 분리.
    * urllib.quote (quote_plus()는 key, value를 따옴표)

In [49]:
!curl http://httpbin.org/encoding/utf8

<h1>Unicode Demo</h1>

<p>Taken from <a
href="http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt">http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt</a></p>

<pre>

UTF-8 encoded sample plain-text file
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾

Markus Kuhn [ˈmaʳkʊs kuːn] <http://www.cl.cam.ac.uk/~mgk25/> — 2002-07-25


The ASCII compatible UTF-8 encoding used in this plain-text file
is defined in Unicode, ISO 10646-1, and RFC 2279.


Using Unicode/UTF-8, you can write in emails and source code things such as

Mathematics and sciences:

  ∮ E⋅da = Q,  n → ∞, ∑ f(i) = ∏ g(i),      ⎧⎡⎛┌─────┐⎞⎤⎫
                                            ⎪⎢⎜│a²+b³ ⎟⎥⎪
  ∀x∈ℝ: ⌈x⌉ = −⌊−x⌋, α ∧ ¬β = ¬(¬α ∨ β),    ⎪⎢⎜│───── ⎟⎥⎪
                                            ⎪⎢⎜⎷ c₈   ⎟⎥⎪
  ℕ ⊆ ℕ₀ ⊂ ℤ ⊂ ℚ ⊂ ℝ ⊂ ℂ,                   ⎨⎢⎜       ⎟⎥⎬
                                            ⎪⎢⎜ ∞     ⎟⎥⎪
  ⊥ < a ≠ b ≡ c ≤ d ≪ ⊤ ⇒ (⟦A⟧ ⇔ ⟪B⟫),      ⎪⎢⎜ ⎲     ⎟⎥⎪
                                         

* curl로 처리하면 결과를 볼 수 있지만, python은 unicode 문자열을 가지고 있다.

In [58]:
r = requests.get("http://httpbin.org/encoding/utf8")

* 맨 앞 u는 unicode라는 의미, 따옴표 안의 문자열이 유니코드이다.

In [63]:
r.text[0:500]

u'<h1>Unicode Demo</h1>\n\n<p>Taken from <a\nhref="http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt">http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt</a></p>\n\n<pre>\n\nUTF-8 encoded sample plain-text file\n\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\u203e\n\nMarkus Kuhn [\u02c8ma\u02b3k\u028as ku\u02d0n] <http://www.cl.cam.ac.uk/~mgk25/> \u2014 2002-07-25\n\n\nThe ASCII compatible UTF-8 encoding used in this plain-text file\nis defined in Unicode, ISO 10646-1, and RFC 2279.\n\n\nUsing Unicode/UTF-8, you can write in emails and so'

In [64]:
teststr=r.text[0:500]

* python print문은 시스템에서 사용하는 encoding을 사용한다.
    * locale은 utf-8을 사용하므로, 출력하면 unicode를 이해하고 출력한다.

In [71]:
import locale
locale.getdefaultlocale()

('en_US', 'UTF-8')

In [72]:
print teststr

<h1>Unicode Demo</h1>

<p>Taken from <a
href="http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt">http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt</a></p>

<pre>

UTF-8 encoded sample plain-text file
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾

Markus Kuhn [ˈmaʳkʊs kuːn] <http://www.cl.cam.ac.uk/~mgk25/> — 2002-07-25


The ASCII compatible UTF-8 encoding used in this plain-text file
is defined in Unicode, ISO 10646-1, and RFC 2279.


Using Unicode/UTF-8, you can write in emails and so


### python의 인코딩

* Python 3에서는 default to utf-8 (when LC_CTYPE is unicode-aware)

* Python의 print문은 기본 defaultencoding을 사용한다.

In [18]:
import sys
print sys.getdefaultencoding()
print sys.stdin.encoding
print sys.stdout.encoding

ascii
None
UTF-8


* os locale로 정의되어 있다.
* os env를 보면, LANG이 정의되어 있다.

In [19]:
!locale

LANG=en_US.UTF-8
LANGUAGE=en_US:en
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC=ko_KR.UTF-8
LC_TIME=ko_KR.UTF-8
LC_COLLATE="en_US.UTF-8"
LC_MONETARY=ko_KR.UTF-8
LC_MESSAGES="en_US.UTF-8"
LC_PAPER=ko_KR.UTF-8
LC_NAME=ko_KR.UTF-8
LC_ADDRESS=ko_KR.UTF-8
LC_TELEPHONE=ko_KR.UTF-8
LC_MEASUREMENT=ko_KR.UTF-8
LC_IDENTIFICATION=ko_KR.UTF-8
LC_ALL=


In [11]:
import os
os.environ['LANG']

'en_US.UTF-8'

* python의 기본 encoding을 변경한다.

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

* ascii는 unicode로 encoding할 수 없다.

In [12]:
str('한')

'\xed\x95\x9c'

In [13]:
u'한'

u'\ud55c'

In [31]:
(u'한').encode('utf-8')

'\xed\x95\x9c'

In [33]:
'한'.encode('utf-8')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xed in position 0: ordinal not in range(128)

* stdout encoding 'utf-8'에 따라 출력한다.

In [73]:
print u'한'

한


### URL 인코딩

* 인코딩이 필요 없는 문자
    ```
    A-Z a-z 0-9 - _ . ~
    ```

* 인코딩이 필요한 문자
    ```
    ! * ' ( ) ; : @ & = + $ , / ? % # [ ]
    ```

* urllib.quote()
    * URL을 부호화할 경우, 특수문자를 %xx 형식으로 표현 (문자, 숫자, '_', '.', '-' 제외)
    * safe문자를 추가로 정의할 수 있다 
    ```
    urllib.quote('hello~@#$', safe='~@#$&()*!+=:;,.?/\'');
    ```

* urllib.quote_plus()
    * urllib.quote()과 다른 점은 'spaces by plus signs'

In [2]:
import urllib

mystr = u"한"
utf8str=mystr.encode('utf-8')  # error without encoding
print urllib.quote(utf8str)
print urllib.quote_plus(utf8str)

%ED%95%9C
%ED%95%9C


In [3]:
print urllib.quote_plus(' ')
print urllib.quote(' ')

+
%20


In [7]:
urllib.quote_plus('%#')

'%25%23'

In [8]:
urllib.quote('%#')

'%25%23'

* 물결표시 '~'가 %xx형식으로 변환된다.

In [31]:
urllib.quote('/~connolly/')

'/%7Econnolly/'

In [32]:
urllib.quote_plus('/~connolly/')

'%2F%7Econnolly%2F'

### percent encoding 문제

* urlencode는 utf-8에서 사용하는 '%'를 '%25'로 encoding하는 문제
* 공공데이터포털 utf-8로 변환된 키 값은 urlencode()를 하지 않는다.

In [11]:
import urllib
params=urllib.urlencode({'key':'%3D'})
print params

k=%253D
k=%253D


## R.5 XML

### XML 소개

* Extensible Markup Language (XML). html과 같은 마크업 언어.
* element는 (1) 루트 element (맨 처음 나오는 element) + (2) 루트 아래에 있는 elements로 계층적으로 구성
    * start tag $<$foo$>$
    * end tag $<$/foo$>$
* nesting element
* root 한 개만 허용. 아래와 같이 2개를 사용하면 xml이 아님.
    ```
    <kim></kim>
    <lee></lee>
    ```
    
* attributes
    * 여러 속성은 공백으로 구분
    * key=value 짝으로 구성
    * start tag안에 적음
    * value는 따옴표
    ```
    <kim x='1'>I am kim</kim>
    ```
    
* xml header
    * version - 1.0과 같이 버전 정보를 넣는다.
    * encoding - utf-8을 지정할 수 있다.
    * root 앞에 위치한다.
    ```
    <?xml version='1.0' encoding='utf-8'?>
    ```
    
* parser
    * built-in - ElementTree
    * larger xml - lxml (따로 설치해야 함)

### XML 읽기

* ElementTree를 사용하여 xml데이터 처리
* string에서 읽거나, 파일에서 읽어 온다. 루트는 getroot() 함수를 이용함.

구분 | string | file
---|---|---
읽기 | ET.fromstring() | ET.parse()
root | 읽으면 root | getroot()를 사용해서 root를 얻음

In [1]:
input = '''<students>
<student x="1">
    <id>001</id>
    <name>Kim</name>
</student>
<student x="2">
    <id>002</id>
    <name>Lee</name>
</student>
</students>
'''
print input
import xml.etree.ElementTree as ET
nodes=ET.fromstring(input)
for node in nodes.getiterator():
    print node.tag, node.attrib


<students>
<student x="1">
    <id>001</id>
    <name>Kim</name>
</student>
<student x="2">
    <id>002</id>
    <name>Lee</name>
</student>
</students>

students {}
student {'x': '1'}
id {}
name {}
student {'x': '2'}
id {}
name {}


In [2]:
%%writefile src/ds_rest_hello.xml
<students>
<student x="1">
    <id>001</id>
    <name>Kim</name>
</student>
<student x="2">
    <id>002</id>
    <name>Lee</name>
</student>
</students>

Writing src/ds_rest_hello.xml


In [5]:
xml=ET.parse('src/ds_rest_hello.xml')
root=xml.getroot()
for node in root.getiterator():
    print node.tag, node.attrib


students {}
student {'x': '1'}
id {}
name {}
student {'x': '2'}
id {}
name {}


In [6]:
std=root.find('student')
for node in std:
    print node.text

001
Kim


In [8]:
stds=root.findall('student')
for node in stds:
    for item in node:
        print item.text

001
Kim
002
Lee


In [9]:
print std.attrib
print stds[1].attrib
std.tag

{'x': '1'}
{'x': '2'}


'student'

## R.6 json

* json (JavaScript Object Notation)은 javascript에서 사용하는 표기
* python의 dictionary형식과 같다.

In [65]:
import json
input = '''[
    { "id" : "001", "x" : "2", "name" : "Chuck"},
    { "id" : "009","x" : "7","name" : "Brent" }
]'''

In [66]:
info = json.loads(input)
print 'User count:', len(info)

for item in info:
    print item['id'], item['name']

User count: 2
001 Chuck
009 Brent


In [67]:
import json
json.dumps(
    ['foo', {'bar': ('baz', None, 1.0, 2)}]
)

'["foo", {"bar": ["baz", null, 1.0, 2]}]'

## 문제 REST-1: ip주소에서 지역정보 알아내기

* freegeoip.net은 ip주소의 국가, 위경도, 지역을 조회

In [19]:
import requests
url='http://freegeoip.net/json/'
geostr=requests.get(url).text
print geostr

{"ip":"117.16.44.45","country_code":"KR","country_name":"Republic of Korea","region_code":"","region_name":"","city":"","zip_code":"","time_zone":"Asia/Seoul","latitude":37.5112,"longitude":126.9741,"metro_code":0}



* 읽어온 데이터가 string, unicode이다.
* json.loads()를 하면, unicode -> dictionary로 만들어 줌.

In [16]:
type(geostr)

unicode

In [20]:
import json
geojson=json.loads(geostr)

In [27]:
type(geojson)

dict

In [21]:
print geojson['ip']

117.16.44.45


* 앞에 보이는 u는 유니코드라는 의미.

In [22]:
geojson.get('ip')

u'117.16.44.45'

In [25]:
country=geojson.get('country_code')

In [26]:
print country.decode('utf-8')

KR


* getCountry()함수 만들어보기
    * urllib라이브러리 urlopen.read()로 읽어온다.
    * json자료에서 country_code key를 찾는다.

In [10]:
import json
import urllib

def getCountry(ipAddress):
    response = urllib.urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get("country_code")

print(getCountry("39.118.87.152"))

KR


* requests.get(url)은 url에서 페이지를 가져오기

In [26]:
import requests
send_url = 'http://freegeoip.net/json/39.118.87.152'
r = requests.get(send_url)

In [47]:
j=json.loads(r.text)

In [48]:
type(j)

dict

In [49]:
print j.keys()

[u'city', u'region_code', u'region_name', u'ip', u'time_zone', u'longitude', u'metro_code', u'latitude', u'country_code', u'country_name', u'zip_code']


In [50]:
print j['city']

Seoul


In [61]:
for k,v in j.iteritems():
    print k,"\t: ",v

city 	:  Seoul
region_code 	:  11
region_name 	:  Seoul
ip 	:  39.118.87.152
time_zone 	:  Asia/Seoul
longitude 	:  126.9783
metro_code 	:  0
latitude 	:  37.5985
country_code 	:  KR
country_name 	:  Republic of Korea
zip_code 	:  


## 문제 REST-2: 키를 저장하고 읽기

* 사용자id, 비밀번호를 사용하는 방식은 보안에 취약하다.
* OAuth를 사용하여 제3자에게 접근권한을 부여하는 방법이 보다 안전하다.
* 파일에 안전하게 저장하여, '비밀번호'와 같은 보안내용이 프로그램에 직접 사용되지 않도록 주의한다.

In [5]:
def getKey(keyPath):
    d=dict()
    f=open(keyPath,'r')
    for line in f.readlines():
        row=line.split('=')
        row0=row[0]
        d[row0]=row[1].strip()
    return d

In [6]:
import os

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=getKey(keyPath)

## 문제 REST-3: 서울시 노선별 지하철역 목록 구하기

* '서울시 노선별 지하철역 정보' - 호선, 역 명, 외부에 지하철 역 이름과 같이 적혀 있는 역 번호
* 샘플URL

호선 | 샘플URL
-----|-----
1호선 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/SearchSTNBySubwayLineService/1/5/1/
2호선 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/SearchSTNBySubwayLineService/1/5/2/
5호선 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/SearchSTNBySubwayLineService/1/5/5/

* 호선
1~9: 1~9호선, I: 인천1호선, K: 경의중앙선, B: 분당선, A: 공항철도, G: 경춘선, S:신분당선, SU:수인선

* params 앞의 32자리 비밀번호는 제외하고 출력한다.
* 페이징을 하려면 INDEX를 조정한다.

In [125]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='SearchSTNBySubwayLineService'
START_INDEX=str(1)
END_INDEX=str(10)
LINE_NUM=str(2)

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
print params[31:]

xml/SearchSTNBySubwayLineService/1/10/2


* 사례에는 모두 동일한 url이 사용된다. 'http://openAPI.seoul.go.kr:8088/'

In [126]:
import urlparse
_url='http://openAPI.seoul.go.kr:8088/'
url=urlparse.urljoin(_url,params)
#print url

In [127]:
import requests
data=requests.get(url).text
print data

<?xml version="1.0" encoding="UTF-8"?><SearchSTNBySubwayLineService>
<list_total_count>51</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<STATION_CD>0201</STATION_CD>
<STATION_NM>시청</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>201</FR_CODE>
</row>
<row>
<STATION_CD>0202</STATION_CD>
<STATION_NM>을지로입구</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>202</FR_CODE>
</row>
<row>
<STATION_CD>0203</STATION_CD>
<STATION_NM>을지로3가</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>203</FR_CODE>
</row>
<row>
<STATION_CD>0204</STATION_CD>
<STATION_NM>을지로4가</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>204</FR_CODE>
</row>
<row>
<STATION_CD>0205</STATION_CD>
<STATION_NM>동대문역사문화공원</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>205</FR_CODE>
</row>
<row>
<STATION_CD>0206</STATION_CD>
<STATION_NM>신당</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>206</FR_CODE>
</row>
<row>
<STATION_CD>0207</STATION_CD>
<STATION_NM>상왕십리</STATION_NM>
<LINE_NUM>2</LINE_NUM>
<FR_CODE>207<

In [76]:
type(data)

unicode

In [77]:
import re
p=re.compile('<STATION_NM>(.+?)</STATION_NM>')
res=p.findall(data)
for item in res:
    print item

시청
을지로입구
을지로3가
을지로4가
동대문역사문화공원
신당
상왕십리
왕십리
한양대
뚝섬


In [78]:
import xml.etree.ElementTree as ET
tree=ET.fromstring(data.encode('utf-8'))

In [110]:
stds=tree.findall('row')
for elements in stds:
    for elm in elements:
        print elm.text

0201
시청
2
201
0202
을지로입구
2
202
0203
을지로3가
2
203
0204
을지로4가
2
204
0205
동대문역사문화공원
2
205
0206
신당
2
206
0207
상왕십리
2
207
0208
왕십리
2
208
0209
한양대
2
209
0210
뚝섬
2
210


In [128]:
import lxml
import lxml.etree
import StringIO

#tree=lxml.etree.parse(StringIO.StringIO(data.encode('utf-8')))
tree=lxml.etree.fromstring(data.encode('utf-8'))

nodes=tree.xpath('//STATION_NM')
for node in nodes:
    print node.text

시청
을지로입구
을지로3가
을지로4가
동대문역사문화공원
신당
상왕십리
왕십리
한양대
뚝섬


## 문제 REST-4: 서울시 외국인 주민 자녀 국적취득 년도별 시군구 합계 구하기

* '서울시 외국인 주민 자녀 국적취득 현황 정보'
* 샘플URL
http://openapi.seoul.go.kr:8088/(인증키)/xml/ListLocaldata470401S/1/5/

* 년도별 시군구 합계

In [23]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='ListLocaldata470401S'
START_INDEX=str(1)
END_INDEX=str(5)

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX)

print params[31:]

xml/ListLocaldata470401S/1/5


In [130]:
import urlparse
_url='http://openapi.seoul.go.kr:8088/'
url=urlparse.urljoin(_url,params)
#print url

In [131]:
import requests

data=requests.get(url).text
print data

<?xml version="1.0" encoding="UTF-8"?><ListLocaldata470401S>
<list_total_count>319</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<SITUYM>2012</SITUYM>
<CGGNM>서울특별시 양천구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 18세 이하</KDDETAILSENM>
<STATMAN>41</STATMAN>
</row>
<row>
<SITUYM>2013</SITUYM>
<CGGNM>서울특별시 양천구</CGGNM>
<KDDETAILSENM>국적을 미취득한 자녀 만 19세 이상</KDDETAILSENM>
<STATMAN>1</STATMAN>
</row>
<row>
<SITUYM>2014</SITUYM>
<CGGNM>서울특별시 강서구</CGGNM>
<KDDETAILSENM>국적을 미취득한 자녀 만 19세 이상</KDDETAILSENM>
<STATMAN>2</STATMAN>
</row>
<row>
<SITUYM>2014</SITUYM>
<CGGNM>서울특별시 금천구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 18세 이하</KDDETAILSENM>
<STATMAN>1409</STATMAN>
</row>
<row>
<SITUYM>2013</SITUYM>
<CGGNM>서울특별시 금천구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 18세 이하</KDDETAILSENM>
<STATMAN>1397</STATMAN>
</row>
</ListLocaldata470401S>



In [122]:
print type(data.encode('utf-8'))

<type 'str'>


In [132]:
import lxml
import lxml.etree
import StringIO

#tree=lxml.etree.parse(StringIO.StringIO(data.encode('utf-8')))
tree=lxml.etree.fromstring(data.encode('utf-8'))

nodes=tree.xpath('//STATMAN')
for node in nodes:
    print node.text

41
1
2
1409
1397


## 문제 REST-5: 2015년 서울시 지하철역별 월별 승하차인원 구하기

* '서울시 지하철역별 승하차인원'
    * 월별 지하철역별 승하차 인원

* 샘플URL 2013년 6월의 지하철역 승하차인원
http://openAPI.seoul.go.kr:8088/(인증키)/xml/CardSubwayStatisticsService/1/5/201306/


In [ ]:
import os
import requests
_url='http://openAPI.seoul.go.kr:8088'
_key=str(key['dataseoul'])
_type='xml'
_service='CardSubwayStatisticsService'
_start_index=1
_end_index=5
_use_mon='201306'
_api=os.path.join(_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon)

In [134]:
response = requests.get(_api).text
print response

<?xml version="1.0" encoding="UTF-8"?><CardSubwayStatisticsService>
<list_total_count>530</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>용문</SUB_STA_NM>
<RIDE_PASGR_NUM>111275</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>108878</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>원덕</SUB_STA_NM>
<RIDE_PASGR_NUM>11495</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>10964</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>양평</SUB_STA_NM>
<RIDE_PASGR_NUM>118103</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>116604</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>오빈</SUB_STA_NM>
<RIDE_PASGR_NUM>10590</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>10020</ALIGHT_PASGR_NUM>
<WORK_DT>2013072

## 문제 REST-6: 2013년 1년 동안 지하철역 승하차 인원

* 2013년 1년 동안 호선별 승하차 인원을 계산한다.
* 페이징
    * index를 사용하여 모든 데이터를 가져올 수 있다.
    * 반복문을 사용하여 모든 데이터를 가져올 수 있다. 반복 종료 조건을 사용하려면 메시지를 판단한다.

메시지 | 메시지 설명
------|-------
INFO-000 | 정상 처리되었습니다.
INFO-100 | 인증키가 유효하지 않습니다.
INFO-200 | 해당하는 데이터가 없습니다.


* ds_rest_subwayPassengetrs.py에 저장

In [144]:
!ls -l src/ds_rest_*

-rw-rw-r-- 1 jsl jsl  152 10월  9 07:10 src/ds_rest_hello.xml
-rw-rw-r-- 1 jsl jsl 1391  8월 30 21:09 src/ds_rest_subwayPassengers.py


In [ ]:
import os
import requests
_url='http://openAPI.seoul.go.kr:8088'
_key=str(key['dataseoul'])
_type='xml'
_service='CardSubwayStatisticsService'
_start_index=1
_end_index=5
_use_mon='201306'

In [135]:
_maxIter=2
_iter=0
while _iter<_maxIter:
    _api=os.path.join(_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon)
    #print _api
    response = requests.get(_api).text
    print response
    _start_index+=5
    _end_index+=5
    _iter+=1

<?xml version="1.0" encoding="UTF-8"?><CardSubwayStatisticsService>
<list_total_count>530</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>용문</SUB_STA_NM>
<RIDE_PASGR_NUM>111275</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>108878</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>원덕</SUB_STA_NM>
<RIDE_PASGR_NUM>11495</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>10964</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>양평</SUB_STA_NM>
<RIDE_PASGR_NUM>118103</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>116604</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>오빈</SUB_STA_NM>
<RIDE_PASGR_NUM>10590</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>10020</ALIGHT_PASGR_NUM>
<WORK_DT>2013072

* how to insert into mongodb

In [146]:
type(response)

unicode

### 위 예제의 json 반환

In [137]:
_type='json'

In [138]:
_api=os.path.join(_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon)

In [141]:
data=requests.get(_api).text
print data

{"CardSubwayStatisticsService":{"list_total_count":530,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"USE_MON":"201306","LINE_NUM":"중앙선","SUB_STA_NM":"도심","RIDE_PASGR_NUM":75553.0,"ALIGHT_PASGR_NUM":67667.0,"WORK_DT":"20130723","COMMT":""},{"USE_MON":"201306","LINE_NUM":"중앙선","SUB_STA_NM":"덕소","RIDE_PASGR_NUM":189783.0,"ALIGHT_PASGR_NUM":189419.0,"WORK_DT":"20130723","COMMT":""},{"USE_MON":"201306","LINE_NUM":"중앙선","SUB_STA_NM":"양정","RIDE_PASGR_NUM":27878.0,"ALIGHT_PASGR_NUM":23735.0,"WORK_DT":"20130723","COMMT":""},{"USE_MON":"201306","LINE_NUM":"중앙선","SUB_STA_NM":"도농","RIDE_PASGR_NUM":267402.0,"ALIGHT_PASGR_NUM":228445.0,"WORK_DT":"20130723","COMMT":""},{"USE_MON":"201306","LINE_NUM":"중앙선","SUB_STA_NM":"구리","RIDE_PASGR_NUM":399287.0,"ALIGHT_PASGR_NUM":418036.0,"WORK_DT":"20130723","COMMT":""}]}}


In [104]:
import json
jd = json.loads(data)
#print jd['STATION_NM']
#for key,value in jd.items():
#    print "---",key,value

print jd['SearchSTNBySubwayLineService']['row'][0]
#n=len(jd['SearchSTNBySubwayLineService']['row'])
#for i in range(0,n):
#    print jd['SearchSTNBySubwayLineService']['row'][i]
for item in jd['SearchSTNBySubwayLineService']['row']:
    print item.keys()
    for i in item.keys():
        if i=='STATION_NM':
            print ''.join(item.values())
            print item.values()[1]


{u'STATION_CD': u'0201', u'STATION_NM': u'\uc2dc\uccad', u'FR_CODE': u'201', u'LINE_NUM': u'2'}
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0201시청2012
시청
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0202을지로입구2022
을지로입구
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0203을지로3가2032
을지로3가
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0204을지로4가2042
을지로4가
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0205동대문역사문화공원2052
동대문역사문화공원
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0206신당2062
신당
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0207상왕십리2072
상왕십리
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0208왕십리2082
왕십리
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0209한양대2092
한양대
[u'STATION_CD', u'STATION_NM', u'FR_CODE', u'LINE_NUM']
0210뚝섬2102
뚝섬


## 문제 REST-7: '2013년 1년 동안 지하철역 승하차 인원' 저장하기

* 앞 문제를 python 일괄 실행하기
* 결과 저장하기 - 파일, nosql mongodb로 저장

### python으로 일괄 실행하기

* json 파일 저장 'src/ds_rest_subwayPassengers.json'
* mongo 저장한다. 한글 unicode 지원한다.
    * db명: use ds_rest_subwayPassengers_mongo_db
    * table (또는 Collection)명: db_rest_subway


In [1]:
%%writefile src/ds_rest_subwayPassengers.py

import os
import requests
import json
from pymongo import MongoClient

Client = MongoClient('localhost:27017')
_db=Client['ds_rest_subwayPassengersDb'] # db is created by mongo
_table=_db['db_rest_subwayTable'] # 'db table' is 'collection'
#db=Client.ds_rest_subwayPassengers

def saveJson(_fname,_data):
    import io
    with io.open(_fname, 'a', encoding='utf8') as json_file:
        _j=json.dumps(_data, json_file, ensure_ascii=False, encoding='utf8')
        json_file.write(_j+"\n")

def readJson(_fname):
    for line in open(_fname, 'r').readlines():
        _j=json.loads(line)
        #print _j['id'],_j['text']
        print _j['id']

def saveDB(_data):
    _table.insert_one(_data)

def readDB():
    for tweet in _table.find():
        print tweet['id'],tweet['text']

def saveFile(_fname,_data):
    fp=open(_fname,'a')
    fp.write(_data+"\n")

_url='http://openAPI.seoul.go.kr:8088'
_key='73725974496a736c34395757705847'
_type='json'
_service='CardSubwayStatisticsService'
_start_index=1
_end_index=5
_use_mon='201306'
_maxIter=20
_iter=0
_jfname='src/ds_rest_subwayPassengers.json'
while _iter<_maxIter:
    _api=os.path.join(_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon)
    #print _api
    r=requests.get(_api)
    _json=r.json()
    print _json
    saveJson(_jfname,_json)
    saveDB(_json)
    _start_index+=5
    _end_index+=5
    _iter+=1

Overwriting src/ds_rest_subwayPassengers.py


In [2]:
!python src/ds_rest_subwayPassengers.py

{u'CardSubwayStatisticsService': {u'row': [{u'COMMT': u'', u'RIDE_PASGR_NUM': 111275.0, u'WORK_DT': u'20130723', u'LINE_NUM': u'\uc911\uc559\uc120', u'SUB_STA_NM': u'\uc6a9\ubb38', u'ALIGHT_PASGR_NUM': 108878.0, u'USE_MON': u'201306'}, {u'COMMT': u'', u'RIDE_PASGR_NUM': 11495.0, u'WORK_DT': u'20130723', u'LINE_NUM': u'\uc911\uc559\uc120', u'SUB_STA_NM': u'\uc6d0\ub355', u'ALIGHT_PASGR_NUM': 10964.0, u'USE_MON': u'201306'}, {u'COMMT': u'', u'RIDE_PASGR_NUM': 118103.0, u'WORK_DT': u'20130723', u'LINE_NUM': u'\uc911\uc559\uc120', u'SUB_STA_NM': u'\uc591\ud3c9', u'ALIGHT_PASGR_NUM': 116604.0, u'USE_MON': u'201306'}, {u'COMMT': u'', u'RIDE_PASGR_NUM': 10590.0, u'WORK_DT': u'20130723', u'LINE_NUM': u'\uc911\uc559\uc120', u'SUB_STA_NM': u'\uc624\ube48', u'ALIGHT_PASGR_NUM': 10020.0, u'USE_MON': u'201306'}, {u'COMMT': u'', u'RIDE_PASGR_NUM': 26304.0, u'WORK_DT': u'20130723', u'LINE_NUM': u'\uc911\uc559\uc120', u'SUB_STA_NM': u'\uc544\uc2e0', u'ALIGHT_PASGR_NUM': 26358.0, u'USE_MON': u'201306'}

### 결과의 저장

* json파일
* mongodb

In [6]:
!ls -l src/ds_rest_subway*.json

-rw-rw-r-- 1 jsl jsl 80290 10월  9 21:35 src/ds_rest_subwayPassengers.json


* mongodb는 /var/lib/mongodb에 생성된다.

In [62]:
!ls -l /var/lib/mongodb/ds_*

-rw------- 1 mongodb nogroup 67108864 10월  9 21:35 /var/lib/mongodb/ds_rest_subwayPassengers_mongo_db.0
-rw------- 1 mongodb nogroup 16777216 10월  9 21:35 /var/lib/mongodb/ds_rest_subwayPassengers_mongo_db.ns


* mongo cli
    * db명: use ds_rest_subwayPassengers_mongo_db
    * table (또는 Collection)명: db_rest_subway
    * dictionary 하위 계층은 dot (.)으로 나타낸다.
        ```
        db.db_rest_subway.find({"CardSubwayStatisticsService.row.LINE_NUM":"분당선"})
        ```

In [3]:
%%writefile src/ds_rest_subway_mongo.js
use ds_rest_subwayPassengers_mongo_db
db.db_rest_subway.find({"CardSubwayStatisticsService.row.SUB_STA_NM":"강남구청"})

Overwriting src/ds_rest_subway_mongo.js


In [4]:
!mongo < src/ds_rest_subway_mongo.js

MongoDB shell version: 2.6.10
connecting to: test
switched to db ds_rest_subwayPassengers_mongo_db
{ "_id" : ObjectId("57fa3870f5e6e94359c033f8"), "CardSubwayStatisticsService" : { "row" : [ { "COMMT" : "", "RIDE_PASGR_NUM" : 206631, "WORK_DT" : "20130723", "LINE_NUM" : "분당선", "SUB_STA_NM" : "선정릉", "ALIGHT_PASGR_NUM" : 231260, "USE_MON" : "201306" }, { "COMMT" : "", "RIDE_PASGR_NUM" : 188076, "WORK_DT" : "20130723", "LINE_NUM" : "분당선", "SUB_STA_NM" : "강남구청", "ALIGHT_PASGR_NUM" : 215608, "USE_MON" : "201306" }, { "COMMT" : "", "RIDE_PASGR_NUM" : 345902, "WORK_DT" : "20130723", "LINE_NUM" : "분당선", "SUB_STA_NM" : "압구정로데오", "ALIGHT_PASGR_NUM" : 393142, "USE_MON" : "201306" }, { "COMMT" : "", "RIDE_PASGR_NUM" : 143459, "WORK_DT" : "20130723", "LINE_NUM" : "분당선", "SUB_STA_NM" : "서울숲", "ALIGHT_PASGR_NUM" : 136147, "USE_MON" : "201306" }, { "COMMT" : "", "RIDE_PASGR_NUM" : 180522, "WORK_DT" : "20130723", "LINE_NUM" : "분당선", "SUB_STA_NM" : "수서", "ALIGHT_PASGR_NUM" : 185810, "USE_MON" : "201306"

## 문제 REST-8: 서울 7016번 광화문~상명대까지 2015년 승하차 인원 월별 정류장별 최대 최소 구하기

* 2015년 7016번 정류장별로 승하차인원을 월별로 합계구하기
* '서울시 버스노선별/정류장별 승하차인원 정보'
    * 월별 버스노선별, 정류장별 승하차 인원
* 샘플URL

조건 | 샘플URL
-----|-----
2013년 6월의 버스정류장별 승하차인원 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/CardBusStatisticsService/1/5/201306/
2013년 6월의 7013B 노선에 대한 승하차인원 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/CardBusStatisticsService/1/5/201306/7013B/

* 샘플 실행하기
* regex

In [17]:
import requests
import re
busstopurl='http://openAPI.seoul.go.kr:8088/sample/xml/CardBusStatisticsService/1/5/201510/7016'
data=requests.get(busstopurl).text
print data
p=re.compile('<BUS_STA_NM>(.+?)</BUS_STA_NM>')
res=p.findall(data)
for item in res:
    print item

<?xml version="1.0" encoding="UTF-8"?><CardBusStatisticsService>
<list_total_count>106</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>201510</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>신촌오거리.현대백화점</BUS_STA_NM>
<RIDE_PASGR_NUM>10423</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>11121</ALIGHT_PASGR_NUM>
<WORK_DT>20151108</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201510</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>갈월동4호선숙대입구</BUS_STA_NM>
<RIDE_PASGR_NUM>3912</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>5539</ALIGHT_PASGR_NUM>
<WORK_DT>20151108</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201510</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>원효로1동.주민센터</BUS_STA_NM>
<RIDE_PASGR_NUM>3268</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>2603</ALIGHT_PASGR_NUM>
<WORK_DT>20151108</WORK_DT>
<COMMT/>
</row>
<row

In [16]:
buspassengers='http://openAPI.seoul.go.kr:8088/sample/xml/CardBusStatisticsService/1/5/201510/7016/'
data=requests.get(busstopurl).text
print data
p=re.compile('<RIDE_PASGR_NUM>(.+?)</RIDE_PASGR_NUM>')
res=p.findall(data)
for item in res:
    print item

<?xml version="1.0" encoding="UTF-8"?><CardBusStatisticsService>
<list_total_count>106</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>201510</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>신촌오거리.현대백화점</BUS_STA_NM>
<RIDE_PASGR_NUM>10423</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>11121</ALIGHT_PASGR_NUM>
<WORK_DT>20151108</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201510</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>갈월동4호선숙대입구</BUS_STA_NM>
<RIDE_PASGR_NUM>3912</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>5539</ALIGHT_PASGR_NUM>
<WORK_DT>20151108</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201510</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>원효로1동.주민센터</BUS_STA_NM>
<RIDE_PASGR_NUM>3268</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>2603</ALIGHT_PASGR_NUM>
<WORK_DT>20151108</WORK_DT>
<COMMT/>
</row>
<row

* 자신의 키로 실행하기

In [15]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='CardBusStatisticsService'
START_INDEX=str(1)
END_INDEX=str(5)
USE_MON=str(201512)
BUS_ROUTE_NO=str(7016)

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,USE_MON,BUS_ROUTE_NO)
print params[31:]

xml/CardBusStatisticsService/1/5/201512/7016


In [22]:
import urlparse
_url='http://openapi.seoul.go.kr:8088/'
url=urlparse.urljoin(_url,params)
#print url

In [17]:
import requests

data=requests.get(url).text
print data

<?xml version="1.0" encoding="UTF-8"?><CardBusStatisticsService>
<list_total_count>106</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>201512</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>신촌오거리.현대백화점</BUS_STA_NM>
<RIDE_PASGR_NUM>10416</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>11279</ALIGHT_PASGR_NUM>
<WORK_DT>20160308</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201512</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>갈월동4호선숙대입구</BUS_STA_NM>
<RIDE_PASGR_NUM>3968</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>4525</ALIGHT_PASGR_NUM>
<WORK_DT>20160308</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201512</USE_MON>
<BUS_ROUTE_NO>7016</BUS_ROUTE_NO>
<BUS_ROUTE_NM>7016번(은평차고지~상명대)</BUS_ROUTE_NM>
<BUS_STA_NM>원효로1동.주민센터</BUS_STA_NM>
<RIDE_PASGR_NUM>3582</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>2481</ALIGHT_PASGR_NUM>
<WORK_DT>20160308</WORK_DT>
<COMMT/>
</row>
<row

## 문제 REST-9: 서울시 종로구 관측소 현재 온도, 습도, 강수량 구하기

* 서울시 관측소별 실시간 기상관측 현황
* 샘플URL

조건 | 샘플URL
-----|-----
관측소별 실시간 기상관측정보 조회 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/RealtimeWeatherStation/1/5/
중구 관측소 실시간 기상관측정보 조회 | http://openAPI.seoul.go.kr:8088/(인증키)/xml/RealtimeWeatherStation/1/5/중구

* 결과
    ```
    1	SAWS_OBS_TM	측정일시
    2	STN_NM	지점명
    3	SAWS_TA_AVG	기온(℃)
    4	SAWS_HD	습도(%)
    5	CODE	풍향1
    6	NAME	풍향2
    7	SAWS_WS_AVG	풍속(m/s)
    8	SAWS_RN_SUM	강수(mm)
    9	SAWS_SOLAR	일사(Mj/㎡)
    10	SAWS_SHINE	일조(hour)
    ```

* 샘플 실행하기
* regex

In [3]:
import requests
import re
weatherurl='http://openAPI.seoul.go.kr:8088/sample/xml/RealtimeWeatherStation/1/5/'
data=requests.get(weatherurl).text
print data
p=re.compile('<SAWS_TA_AVG>(.+?)</SAWS_TA_AVG>')
res=p.findall(data)
for item in res:
    print item

<?xml version="1.0" encoding="UTF-8"?><RealtimeWeatherStation>
<list_total_count>1</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<SAWS_OBS_TM>2015092216</SAWS_OBS_TM>
<STN_NM>성동</STN_NM>
<SAWS_TA_AVG>30.1</SAWS_TA_AVG>
<SAWS_HD>29.2</SAWS_HD>
<CODE>7.0</CODE>
<NAME>남동</NAME>
<SAWS_WS_AVG>1.9</SAWS_WS_AVG>
<SAWS_RN_SUM>0</SAWS_RN_SUM>
<SAWS_SOLAR/>
<SAWS_SHINE/>
</row>
</RealtimeWeatherStation>

30.1


* 자신의 키로 실행하기

In [18]:
KEY=str(key['dataseoul'])
TYPE='json'
SERVICE='RealtimeWeatherStation'
START_INDEX=str(1)
END_INDEX=str(5)
STN_NM=u'중구'

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STN_NM)
print params[31:]

json/RealtimeWeatherStation/1/5/중구


In [19]:
import urlparse
_url='http://openapi.seoul.go.kr:8088/'
url=urlparse.urljoin(_url,params)
#print url

In [20]:
import requests

data=requests.get(url).text
print data

{"RealtimeWeatherStation":{"list_total_count":1,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"SAWS_OBS_TM":"2016100919","STN_NM":"중구","SAWS_TA_AVG":13.100000381469727,"SAWS_HD":39.70000076293945,"CODE":"13","NAME":"서","SAWS_WS_AVG":1.7999999523162842,"SAWS_RN_SUM":0.0,"SAWS_SOLAR":0.0,"SAWS_SHINE":0.0}]}}


## 문제 REST-10: 공공데이터포털 2015년 서울시 구별 교통사고 사망자 수 구하기

* 예제는 2013년 종로구의 사망자 수를 구한다.
* data.go.kr 공공데이터포털에 회원가입한다.
* 항목 선정
    ```
    재난안전 > 교통사고정보 > 사망교통사고정보서비스 > 활용신청
    ```
    * 활용신청하고자 하는 내역을 입력하고, 활용할 API를 선택하여 '활용신청' 버튼을 클릭한다

* 인증키
    * 1시간이 지나야 사용할 수 있다고 공지해 놓고 있다.
    * 개발 계정 신청이 승인 되었을 경우 인증키를 발급받을수 있다
    * utf-8을 사용한다. 키에는 인코딩 없이는 넘길 수 없는 '=' 같은 특수 문자들이 포함되어서, UTF-8 인코딩 필요하다.
    * 주의
        * urllib.urlencode()를 사용할 경우, 키에 포함된 '%'문자와 같은 특수문자를 주의한다.

* 마이페이지로 가서 신청현황을 볼 수 있다.
    ```
    마이페이지 > 오픈API > 개발계정 
    ```

* '사망교통사고정보서비스' 서비스 예
http://apis.data.go.kr/B552061/trafficAccidentDeath/오퍼레이션명?servicekey=서비스키&searchYearCd=연도코드&sido=법정동시도코드&gugun=법정동시군구코드


* 출력
    * 년월일, 사망자수, 위도, 경도

In [36]:
import os
SERVICE='trafficAccidentDeath'
OPERATION_NAME='getRestTrafficAccidentDeath'
params1=os.path.join(SERVICE,OPERATION_NAME)
print params1

trafficAccidentDeath/getRestTrafficAccidentDeath


In [37]:
import urllib
_d=dict()
_d['searchYearCd']='2014'
_d['sido']='1100' #서울특별시
_d['gugun']='1116'
params2 = urllib.urlencode(_d)
print params2

searchYearCd=2014&gugun=1116&sido=1100


* service key는 '%'와 같이 encoding문제가 발생할 수 있는 문자를 가지고 있다.
* urlencode()에 넣지 않고 별도로 param을 만든다.

In [39]:
params=params1+'?'+'serviceKey='+key['gokr']+'&'+params2
#print params

In [40]:
import urlparse
_url='http://apis.data.go.kr/B552061/'
url=urlparse.urljoin(_url,params)
#print url

In [41]:
import requests

data=requests.get(url).text
#print data

In [42]:
print data[:1000]

{"searchResult":{"accidentDeath":[{"year":"2013","dt_006":"2013071503","dt_006_lv8":"18","cd_008":"2","cd_007":"2","no_010":1,"injpsn_co":1,"no_011":0,"no_012":0,"no_013":0,"cd_003":"1116","cd_003_lv1":"1100","cd_014_lv1":"01","cd_014_lv2":"12","cd_014":"02","cd_027_1_lv1":"01","cd_027_1_lv2":"12","cd_043_lv1":"02","cd_043":"07","cd_036_1_lv1":"01","cd_036_1":"02","cd_036_1_lv2":"11","cd_036_2":"25","x_coord":"960800  ","y_coord":"1946920 ","grd_lo":"127.0563897","grd_la":"37.52074269"},{"year":"2014","dt_006":"2014052704","dt_006_lv8":"55","cd_008":"2","cd_007":"3","no_010":1,"injpsn_co":1,"no_011":0,"no_012":0,"no_013":0,"cd_003":"1116","cd_003_lv1":"1100","cd_014_lv1":"01","cd_014_lv2":"11","cd_014":"01","cd_027_1_lv1":"01","cd_027_1_lv2":"12","cd_043_lv1":"01","cd_043":"05","cd_036_1_lv1":"01","cd_036_1":"02","cd_036_1_lv2":"11","cd_036_2":"25","x_coord":"958125  ","y_coord":"1944706 ","grd_lo":"127.02624489","grd_la":"37.50066957"},{"year":"2013","dt_006":"2013041120","dt_006_lv8"

## 문제 REST-11: 공공데이터포털 대기 오염 정보

* 서비스키는 동일하다.
* '대기오염정보 조회 서비스' 항목을 선택하여 활용신청'한다.

* 예제 (사이트 미리보기)
http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMinuDustFrcstDspth?ServiceKey=


* os.path.join()를 사용하여 경로를 만든다.

In [1]:
import os
SERVICE='ArpltnInforInqireSvc'
OPERATION_NAME='getMinuDustFrcstDspth'
params1=os.path.join(SERVICE,OPERATION_NAME)
print params1

ArpltnInforInqireSvc/getMinuDustFrcstDspth


In [25]:
import urllib
_d=dict()

#_d['stationName']='종로구'
_d['dataTerm']='month'
params2 = urllib.urlencode(_d)
print params2

dataTerm=month


* serviceKey에 대문자 섞임 (위와 다름)

In [33]:
params=params1+'?'+'serviceKey='+key['gokr']+'&'+params2
#print params

In [34]:
import urlparse
_url='http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc'
url=urlparse.urljoin(_url,params)
#print url

* 결과 예
```
<response>
	<header>
		<resultCode>00</resultCode>
		<resultMsg>NORMAL SERVICE.</resultMsg>
	</header>
	<body>
		<items>
				<item>
					<dataTime>2016-10-11</dataTime>
					<informCode>PM25</informCode>
				</item>
			
				<item>
					<dataTime>2016-10-11</dataTime>
					<informCode>O3</informCode>
				</item>
			
		</items>
			<numOfRows>10</numOfRows>
			<pageNo>1</pageNo>
			<totalCount>1062</totalCount>
	</body>
</response>
```

In [35]:
import requests

data=requests.get(url).text
#print data

## 문제 REST-12: 공공데이터포털 서울특별시교육청 공공도서관 소장도서정보


* '서울특별시교육청 공공도서관 소장도서정보'

* manageCd: MA에서 MU까지

코드 | 도서관명
-----|-----
MA | 강남도서관
MU | 종로도서관

* 샘플URL <...>자리에 키(utf-8)를 넣는다.
    ```
    http://openapi-lib.sen.go.kr/openapi/service/lib/openApi?serviceKey=<...>&manageCd=MA&numOfRows=5&pageNo=2&title=%EB%8F%84%EB%91%91
    ```

* 서비스, 오퍼레이션은 별도로 지정하지 않는다.
    ```
    SERVICE=''
    OPERATION_NAME=''
    ```

* title에서는 그대로 한글을 사용하였다. 한글 '도둑'이 url encoding되었다.
* 연습으로 quote_plut()를 사용하였다.

In [49]:
import urllib

_d=dict()
_d['title']='도둑'
_d[quote_plus('manageCd')]='MA'
_d[quote_plus('numOfRows')]='5'
_d[quote_plus('pageNo')]='2'   
params2 = urllib.urlencode(_d)

print params2

manageCd=MA&numOfRows=5&pageNo=2&title=%EB%8F%84%EB%91%91


In [50]:
params='?'+'serviceKey='+key['gokr']+'&'+params2
#print params

In [55]:
import urlparse
_url = 'http://openapi-lib.sen.go.kr/openapi/service/lib/openApi'
url=urlparse.urljoin(_url,params)
# print url

* 사이트 샘플코드 참조
    ```
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    print response_body
    ```

In [52]:
import requests

data=requests.get(url).text
#print data

In [54]:
print data

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><appendixYn>N</appendixYn><author>글·그림: 율리안 프레스 ;옮김: 박선주</author><bookKey>5150656168</bookKey><callNo>아 853-ㅍ82ㄷ-2</callNo><controlNo>KMO201109022</controlNo><isbn>9788965590194</isbn><libName>강남도서관</libName><manageCode>MA</manageCode><pubYear>2011</pubYear><publisher>푸른날개</publisher><regNo>JUA000043858</regNo><rnum>6</rnum><shelfLocName>[강남]어린이실</shelfLocName><speciesKey>5150656166</speciesKey><title>단서를 찾아라!. 2, 보석 도둑과 비밀의 문</title><vol>2</vol><volTitle>보석 도둑과 비밀의 문</volTitle></item><item><appendixYn>N</appendixYn><author>최용주 지음</author><bookKey>5210340360</bookKey><callNo>199.5-ㅊ574ㄱ</callNo><controlNo>KMO201304671</controlNo><isbn>9788960652866</isbn><libName>강남도서관</libName><manageCode>MA</manageCode><pubYear>2013</pubYear><publisher>공감</publisher><regNo>EMA000223336</regNo><rnum>7</rnum><shelfLocName>[강남]종합자료실</shelfL

## 연습 REST: pandas로 웹 데이터 야후 주식 읽기

* pandas data reading from Yahoo Finance
* 파이썬으로 시스템 트레이딩 https://gist.github.com/ridingtide/51a40d87abe1bcfbc5ce
* xing api (이트레이드)
http://nbviewer.ipython.org/github/harpone/PyArb/blob/master/PyArb%20Intro.ipynb?create=1

* Google Finance KOSPI200 -> ds_Web_data

In [1]:
import datetime
import pandas.io.data as web
start = datetime.datetime(2015,11,1)
end = datetime.datetime(2015, 11, 10)
soil = web.DataReader("010950.KS", "yahoo", start, end)

In [2]:
print soil

             Open   High    Low  Close  Volume  Adj Close
Date                                                     
2015-11-02  67900  68400  66800  68400  206300   66836.65
2015-11-03  68600  69800  67600  69700  331900   68106.93
2015-11-04  70900  71500  70000  70300  286900   68693.22
2015-11-05  69800  70000  69100  69600  177600   68009.22
2015-11-06  69500  69600  68900  69500  141000   67911.50
2015-11-09  69300  69300  67600  67600  168800   66054.93
2015-11-10  66700  68300  66200  67300  175600   65761.79


In [3]:
import pandas.io.data as web

ncsoft = web.DataReader('KRX:036570', data_source='google', start='01/01/2015', end='09/02/2015')
sds = web.DataReader('KRX:018260', data_source='google', start='11/01/2015', end='11/10/2015')

print sds

              Open    High     Low   Close  Volume
Date                                              
2015-11-02  257500  259500  254500  255500  153443
2015-11-03  257500  259000  255000  255500  126434
2015-11-04  254500  255000  250500  253000  180937
2015-11-05  253000  256500  250500  254000   85396
2015-11-06  253500  261000  253000  258000   88942
2015-11-09  258500  258500  251000  252500   89752
2015-11-10  251500  252000  248500  250000   79362


## 연습 REST: etc

* 구글 스칼라
    * http://code.activestate.com/recipes/523047-search-google-scholar/
* http://kosis.kr
* 지오코딩 - T Map, 다음, 네이버, 구글 (구글맵에 사용)

In [ ]:
# %load jsldaum.py
# taken from Documents/Code/PCI 20160408FRI
import feedparser
import re
import urllib
# Returns title and dictionary of word counts for an RSS feed
def getwordcounts(url):
  # Parse the feed
  d=feedparser.parse(url)
  wc={}
  # Loop over all the entries
  for e in d.entries:
    if 'summary' in e: summary=e.summary
    else: summary=e.description
    # Extract a list of words
    words=getwords(e.title+' '+summary)
    for word in words:
      wc.setdefault(word,0)
      wc[word]+=1
  return d.feed.title,wc

def getwords(html):
  # Remove all the HTML tags
  txt=re.compile(r'<[^>]+>').sub('',html)
  # Split words by all non-alpha characters
  words=re.compile(r'[^A-Z^a-z]+').split(txt)
  # Convert to lowercase
  return [word.lower( ) for word in words if word!='']

#apcount={}
#wordcounts={}
#for feedurl in file('feedlist1.txt'):
#  title,wc=getwordcounts(feedurl)
#  wordcounts[title]=wc
#  for word,count in wc.items( ):
#    apcount.setdefault(word,0)
#    if count>1:
#     apcount[word]+=1


# -- daum (맨 윗줄 encoding)
# http://dna.daum.net/apis/search/tutorial/basic_python

# simple get
url='http://dna.daum.net'
data=urllib.urlopen(ur).read()
print data


import urllib

url = "http://apis.daum.net/search/knowledge?&;output=xml&q=OpenAPI"

apikey = "DAUM_SEARCH_DEMO_APIKEY"
output = "xml"
q = "OpenAPI"
params = urllib.urlencode({
	'apikey': apikey,
	'output': output,
	'q': q
})
data = urllib.urlopen(url, params).read()
print data
